<a href="https://colab.research.google.com/github/aomidvar/scrapper/blob/main/Cert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import bs4 as bs
import json
import random
from concurrent.futures import ThreadPoolExecutor

from google.colab import drive

drive.mount('/content/drive')

list_of_att = []


def link_extractor(page):
    try:
        r = requests.get('https://www.kb.cert.org/vuls/bypublished/desc/?page='+str(page))
    except requests.exceptions.ProxyError:
        r = requests.get('https://www.kb.cert.org/vuls/bypublished/desc/?page='+str(page),proxies={'http':f'localhost:{random.randint(5002,5016)}','https':f'localhost:{random.randint(5002,5016)}'})
    soup = BeautifulSoup(r.text, 'lxml')
    table = soup.find("table", attrs={"class": "searchby unstriped scroll"})
    links = []
    for table in soup.findAll('table', {'class': 'searchby unstriped scroll'}):
        for tr in table.findAll('tr'):
            for a in tr.findAll('a'):
                if '/vuls/id/' in a['href']:
                    links.append(a['href'])

    return links


def crawl_page(link):
    global list_of_att
    try:
        r = requests.get('https://www.kb.cert.org'+link,proxies={'http':f'localhost:{random.randint(5002,5016)}','https':f'localhost:{random.randint(5002,5016)}'})
    except requests.exceptions.ProxyError:
        r = requests.get('https://www.kb.cert.org' + link, proxies={'http': f'localhost:{random.randint(5002, 5016)}',
                                                                    'https': f'localhost:{random.randint(5002, 5016)}'})
    parsed_article = BeautifulSoup(r.text, "lxml")
    paragraphs = parsed_article.find_all('p')
    headers = parsed_article.find_all('h3')
    title = parsed_article.find('h2', {'class': 'subtitle'})
    dict = {}
    dict['Title'] = title.text
    for header in parsed_article.find_all('h3'):
        para = header.find_next_sibling('p')
        try:
            dict[header.text] = para.text
        except:
            pass

    list_of_att.append(dict)


if __name__ == '__main__':
    for i in range(1,230):
        print(f'page : {i}')
        list_of_link = link_extractor(i)
        with ThreadPoolExecutor(max_workers=15) as executor:
            for link in list_of_link:
                print(link)
                executor.submit(crawl_page,link)

    with open("./content/drive/MyDrive/cert.json", "a", encoding='utf-8') as outfile:
        json.dump(list_of_att, outfile, ensure_ascii=False, indent=4)



ValueError: mount failed

In [ ]:
from bs4 import BeautifulSoup
import requests
r = requests.get('https://www.kb.cert.org/vuls/bypublished/desc/?page=1')
soup = BeautifulSoup(r.text,'lxml')
table = soup.find("table", attrs={"class":"searchby unstriped scroll"})

# The first tr contains the field names.
headings = [th.get_text() for th in table.find("tr").find_all("th")]
print(headings)
#datasets = []
#for row in table.find_all("tr"):
 #   datasets = []
  #  for td in row.find_all("td"):
   #     datasets.append(td.text)
    #print(datasets)
links = []
for table in soup.findAll('table', {'class': 'searchby unstriped scroll'}):
    for tr in table.findAll('tr'):
        for a in tr.findAll('a'):
            if '/vuls/id/' in a['href'] :
                links.append(a['href'])



In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import bs4 as bs
import json

r = requests.get('https://www.kb.cert.org/vuls/id/534195')

parsed_article = BeautifulSoup(r.text, "lxml")
paragraphs = parsed_article.find_all('p')
headers = parsed_article.find_all('h3')
title = parsed_article.find('h2', {'class': 'subtitle'})
dict = {}
dict['Title'] = title.text

table = parsed_article.findAll('div',attrs={"class":"large-12 columns"})
for x in table:
    print (x.find_all('p').text)